In [ ]:
!pip install tvdatafeed
!pip install tensorflow
!pip install scikit-learn

ERROR: Could not find a version that satisfies the requirement tvdatafeed (from versions: none)
ERROR: No matching distribution found for tvdatafeed


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/My\ Drive/Colab\ Notebooks/

Mounted at /content/drive
 AItinyfoodrecog	   'Copy of GNN.ipynb'	        GNNs			'WiDs Investron'
 cGAN			   'Copy of Welcome To Colab'   ProjectAiFoodRecogn
'Copy of AItinyfoodrecog'   data		       ' ProjectAiFoodRecogn2'


In [ ]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/
import os

directory = "/content/drive/My Drive/Colab Notebooks/data/"

if os.path.exists(directory):
    print("Directory exists!")
    print("Files in directory:", os.listdir(directory))
else:
    print("Directory does not exist. Check the path.")


 AItinyfoodrecog	   'Copy of GNN.ipynb'	        GNNs			'WiDs Investron'
 cGAN			   'Copy of Welcome To Colab'   ProjectAiFoodRecogn
'Copy of AItinyfoodrecog'   data		       ' ProjectAiFoodRecogn2'
Directory exists!
Files in directory: ['DRREDDY.csv', 'M&M.csv', 'INDUSINDBK.csv', 'EICHERMOT.csv', 'WIPRO.csv', 'BAJFINANCE.csv', 'TATAMOTORS.csv', 'BAJAJFINSV.csv', 'CIPLA.csv', 'SUNPHARMA.csv', 'TRENT.csv', 'TECHM.csv', 'ICICIBANK.csv', 'NESTLEIND.csv', 'INFY.csv', 'ITC.csv', 'ASIANPAINT.csv', 'MARUTI.csv', 'RELIANCE.csv', 'HDFCBANK.csv', 'HINDUNILVR.csv', 'KOTAKBANK.csv', 'ADANIENT.csv', 'NTPC.csv', 'AXISBANK.csv', 'BRITANNIA.csv', 'TCS.csv', 'BHARTIARTL.csv', 'GRASIM.csv', 'JSWSTEEL.csv', 'APOLLOHOSP.csv', 'POWERGRID.csv', 'TITAN.csv', 'SBILIFE.csv', 'ULTRACEMCO.csv', 'HCLTECH.csv', 'LT.csv', 'BPCL.csv', 'HDFCLIFE.csv', 'HEROMOTOCO.csv', 'TATASTEEL.csv', 'ADANIPORTS.csv', 'BEL.csv', 'SHRIRAMFIN.csv', 'ONGC.csv', 'SBIN.csv', 'COALINDIA.csv', 'MockBroker.py', 'HINDALCO.csv', 'Investron.py'

In [1]:
# Required Libraries
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define Data Directory
data_dir = "/content/drive/My Drive/Colab Notebooks/data/"

# Load Data into Dictionary of DataFrames
def load_data(data_dir):
    dataframe_dict = {}
    for file in os.listdir(data_dir):
        if file.endswith(".csv"):  # Ensure only CSV files are considered
            symbol = file.split(".")[0]  # Extract symbol from filename
            dataframe_dict[symbol] = pd.read_csv(os.path.join(data_dir, file))
    return dataframe_dict

dataframe_dict = load_data(data_dir)

# Splitting Data into Train and Test Sets
train_days = 900
sequence_length = 30  # Number of days in each sequence

def split_data(df, train_days):
    train_data = df.iloc[:train_days]
    test_data = df.iloc[train_days:]
    return train_data, test_data

# Prepare Sequences for LSTM
def create_sequences(data, sequence_length, feature_columns, target_column):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data.iloc[i:i + sequence_length][feature_columns].values)
        y.append(data.iloc[i + sequence_length][target_column])
    return np.array(X), np.array(y)

# Normalize Data
scaler = MinMaxScaler()

# Define LSTM Model
def create_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='tanh')  # Output: -1 to 1
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train Models and Predict for Each Company
predictions = {}
test_data_dict = {}
for symbol, df in dataframe_dict.items():
    print(f"Processing {symbol}...")
    train_data, test_data = split_data(df, train_days)
    test_data_dict[symbol] = test_data.reset_index(drop=True)

    feature_columns = ['MACD', 'RSI', 'close']  # Features for LSTM
    target_column = 'positions'  # Target: -1, 0, 1 (sell, hold, buy)

    X_train, y_train = create_sequences(train_data, sequence_length, feature_columns, target_column)
    X_test, y_test = create_sequences(test_data, sequence_length, feature_columns, target_column)

    # Scale Data
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

    # Train Model
    model = create_model((sequence_length, len(feature_columns)))
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

    # Predict on Test Data
    y_pred = model.predict(X_test)
    y_pred = np.where(y_pred > 0, 1, np.where(y_pred < 0, -1, 0))  # Convert to discrete signals
    predictions[symbol] = y_pred
    # Convert predictions to actionable signals
    threshold_buy = 0.5  # You can adjust these thresholds
    threshold_sell = -0.5

    # Generate buy (1), sell (-1), hold (0) actions
    actions = np.where(y_pred > threshold_buy, 1, np.where(y_pred < threshold_sell, -1, 0))
    predictions[symbol] = actions  # Assign processed actions to predictions


# MockBroker Class
class MockBroker:
    def __init__(self, initial_balance: float, price_data: dict):
        """
        Initialize MockBroker with starting balance and price dictionary for multiple companies.

        Args:
            initial_balance: Starting cash balance
            price_data: Dictionary with stock symbols as keys and their respective DataFrame of prices as values
        """
        self.balance = initial_balance
        self.price_data = price_data
        self.holdings = {symbol: 0 for symbol in price_data.keys()}  # Initialize holdings for all companies
        self.current_day = 0

    def get_current_price(self, symbol: str) -> float:
        """Get the current price for a symbol based on current_day"""
        return self.price_data[symbol].iloc[self.current_day]['close']

    def buy(self, symbol: str, quantity: int) -> bool:
        """Buy specified quantity of a stock if sufficient balance exists"""
        cost = self.get_current_price(symbol) * quantity
        if cost > self.balance:
            return False
        self.balance -= cost
        self.holdings[symbol] += quantity
        return True

    def sell(self, symbol: str, quantity: int) -> bool:
        """Sell specified quantity of a stock if sufficient holdings exist"""
        if self.holdings[symbol] < quantity:
            return False
        proceeds = self.get_current_price(symbol) * quantity
        self.balance += proceeds
        self.holdings[symbol] -= quantity
        return True

    def get_net_worth(self) -> float:
        """Calculate total net worth (cash + value of holdings)"""
        holdings_value = sum(
            self.get_current_price(symbol) * quantity
            for symbol, quantity in self.holdings.items()
        )
        return self.balance + holdings_value

    def next_day(self):
        """Advance to the next trading day"""
        self.current_day += 1

# Simulate Trading for All Companies
broker = MockBroker(10000, test_data_dict)

for i in range(len(test_data_dict['HINDALCO'])):  # Assume all companies have same test days
    for symbol in predictions.keys():
        broker.current_day = i
        action = predictions[symbol][i] if i < len(predictions[symbol]) else 0
        print(f"Day {i}, Symbol: {symbol}, Action: {action}")
        if action == 1:  # Buy
            broker.buy(symbol, 10)
        elif action == -1:  # Sell
            broker.sell(symbol, 10)

# Final Results
print(f"Final Balance: ${broker.balance:.2f}")
print(f"Final Net Worth: ${broker.get_net_worth():.2f}")


Mounted at /content/drive
Processing DRREDDY...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Processing M&M...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
Processing INDUSINDBK...


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Processing COALINDIA...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Processing HINDALCO...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Day 0, Symbol: DRREDDY, Action: [0]
Day 0, Symbol: M&M, Action: [0]
Day 0, Symbol: INDUSINDBK, Action: [0]
Day 0, Symbol: EICHERMOT, Action: [0]
Day 0, Symbol: WIPRO, Action: [0]
Day 0, Symbol: BAJFINANCE, Action: [0]
Day 0, Symbol: TATAMOTORS, Action: [0]
Day 0, Symbol: BAJAJFINSV, Action: [0]
Day 0, Symbol: CIPLA, Action: [0]
Day 0, Symbol: SUNPHARMA, Action: [0]
Day 0, Symbol: TRENT, Action: [0]
Day 0, Symbol: TECHM, Action: [0]
Day 0, Symbol: ICICIBANK, Action: [0]
Day 0, Symbol: NESTLEIND, Action: [0]
Day 0, Symbol: INFY, Action: [0]
Day 0, Symbol: ITC, Action: [0]
Day 0, Symbol: ASIANPAINT, Action: [0]
Day 0, Symbol: MARUTI, Action: [0]
Day 0, Symbol: RELIANCE, Action: [0]
Day 0, Symbol: HDFCBANK, Action: [0]
Day 0, Symbol: HINDUNILVR, Action: [0]
Day 0, Symbol: KOTAKBANK, Action: [0]
Day 0, Symbol: ADANIENT, Action: [0]
Day 0, Symbol: NTPC, Action: [0]
Day 0, Symbol: AXISBANK, Action: [0]
Day 0, Symbol: BRITANNIA, Action: [0]
Day 0, Symbol: 

In [ ]:
# Required Libraries
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
import tensorflow as tf

# Mount Google Drive
drive.mount('/content/drive')

# Define Data Directory
DATA_DIR = "/content/drive/My Drive/Colab Notebooks/data/"

class DataLoader:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.scaler = MinMaxScaler()

    def load_data(self):
        dataframe_dict = {}
        for file in os.listdir(self.data_dir):
            if file.endswith(".csv"):
                symbol = file.split(".")[0]
                df = pd.read_csv(os.path.join(self.data_dir, file))
                print(df)
                # Ensure required columns exist
                required_columns = ['MACD', 'RSI', 'close', 'positions']
                if not all(col in df.columns for col in required_columns):
                    raise ValueError(f"Missing required columns in {file}")
                dataframe_dict[symbol] = df
        return dataframe_dict

    def prepare_data(self, df, train_days, sequence_length):
        # Split data
        train_data = df.iloc[:train_days]
        test_data = df.iloc[train_days:]

        feature_columns = ['MACD', 'RSI', 'close']
        target_column = 'positions'

        # Create sequences
        X_train, y_train = self._create_sequences(train_data, sequence_length, feature_columns, target_column)
        X_test, y_test = self._create_sequences(test_data, sequence_length, feature_columns, target_column)

        # Scale data
        X_train_scaled = self.scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_test_scaled = self.scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

        return X_train_scaled, y_train, X_test_scaled, y_test, test_data.reset_index(drop=True)

    def _create_sequences(self, data, sequence_length, feature_columns, target_column):
        X, y = [], []
        for i in range(len(data) - sequence_length):
            X.append(data.iloc[i:i + sequence_length][feature_columns].values)
            y.append(data.iloc[i + sequence_length][target_column])
        return np.array(X), np.array(y)

class TradingModel:
    def __init__(self, sequence_length, n_features):
        self.model = self._create_model((sequence_length, n_features))

    def _create_model(self, input_shape):
        # Enable mixed precision training for better performance
        tf.keras.mixed_precision.set_global_policy('mixed_float16')

        model = Sequential([
            LSTM(64, input_shape=input_shape, return_sequences=True),
            Dropout(0.2),
            LSTM(64, return_sequences=False),
            Dense(32, activation='relu'),
            Dropout(0.2),
            Dense(1, activation='tanh')
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def train(self, X_train, y_train, epochs=20, batch_size=32):
        # Add early stopping to prevent overfitting
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=5,
            restore_best_weights=True
        )

        self.model.fit(
            X_train,
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping],
            validation_split=0.1,
            verbose=1
        )

    def predict(self, X_test, threshold_buy=0, threshold_sell=0):
        y_pred = self.model.predict(X_test, verbose=0)
        print(y_pred)
        return np.where(y_pred > threshold_buy, 1, np.where(y_pred < threshold_sell, -1, 0))

class MockBroker:
    def __init__(self, initial_balance: float, price_data: dict):
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.price_data = price_data
        self.holdings = {symbol: 0 for symbol in price_data.keys()}
        self.current_day = 0
        self.transaction_history = []

    def get_current_price(self, symbol: str) -> float:
        return self.price_data[symbol].iloc[self.current_day]['close']

    def buy(self, symbol: str, quantity: int) -> bool:
        cost = self.get_current_price(symbol) * quantity
        if cost > self.balance:
            return False
        self.balance -= cost
        self.holdings[symbol] += quantity
        self.transaction_history.append({
            'day': self.current_day,
            'symbol': symbol,
            'action': 'buy',
            'quantity': quantity,
            'price': self.get_current_price(symbol)
        })
        return True

    def sell(self, symbol: str, quantity: int) -> bool:
        if self.holdings[symbol] < quantity:
            return False
        proceeds = self.get_current_price(symbol) * quantity
        self.balance += proceeds
        self.holdings[symbol] -= quantity
        self.transaction_history.append({
            'day': self.current_day,
            'symbol': symbol,
            'action': 'sell',
            'quantity': quantity,
            'price': self.get_current_price(symbol)
        })
        return True

    def get_net_worth(self) -> float:
        holdings_value = sum(
            self.get_current_price(symbol) * quantity
            for symbol, quantity in self.holdings.items()
        )
        return self.balance + holdings_value

    def get_performance_metrics(self):
        final_worth = self.get_net_worth()
        total_return = ((final_worth - self.initial_balance) / self.initial_balance) * 100
        return {
            'initial_balance': self.initial_balance,
            'final_balance': self.balance,
            'final_net_worth': final_worth,
            'total_return_percent': total_return,
            'total_trades': len(self.transaction_history)
        }

    def next_day(self):
        self.current_day += 1

def main():
    # Initialize components
    data_loader = DataLoader(DATA_DIR)
    dataframe_dict = data_loader.load_data()

    # Training parameters
    train_days = 900
    sequence_length = 30

    # Store predictions and test data
    predictions = {}
    test_data_dict = {}

    # Train models for each symbol
    for symbol, df in dataframe_dict.items():
        print(f"\nProcessing {symbol}...")

        # Prepare data
        X_train, y_train, X_test, y_test, test_data = data_loader.prepare_data(
            df, train_days, sequence_length
        )
        test_data_dict[symbol] = test_data

        # Create and train model
        model = TradingModel(sequence_length, X_train.shape[-1])
        model.train(X_train, y_train)

        # Generate predictions
        predictions[symbol] = model.predict(X_test)

    # Initialize and run broker simulation
    broker = MockBroker(10000, test_data_dict)

    # Simulation loop
    print("\nStarting trading simulation...")
    for i in range(len(next(iter(test_data_dict.values())))):
        for symbol in predictions.keys():
            broker.current_day = i
            if i < len(predictions[symbol]):
                action = predictions[symbol][i]
                if action == 1:
                    broker.buy(symbol, 10)
                elif action == -1:
                    broker.sell(symbol, 10)

    # Print final results
    metrics = broker.get_performance_metrics()
    print("\nTrading Simulation Results:")
    print(f"Initial Balance: ${metrics['initial_balance']:,.2f}")
    print(f"Final Balance: ${metrics['final_balance']:,.2f}")
    print(f"Final Net Worth: ${metrics['final_net_worth']:,.2f}")
    print(f"Total Return: {metrics['total_return_percent']:.2f}%")
    print(f"Total Trades: {metrics['total_trades']}")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
          symbol     open     high      low    close      volume       MACD  \
0    NSE:DRREDDY  1010.00  1026.90  1006.00  1014.57   6803005.0   0.000000   
1    NSE:DRREDDY  1016.00  1052.57  1015.63  1045.75  18525665.0   2.487293   
2    NSE:DRREDDY  1048.60  1054.84   992.41  1013.71   9061285.0   1.851785   
3    NSE:DRREDDY  1021.54  1035.00   987.27  1032.36   7764120.0   2.820526   
4    NSE:DRREDDY  1032.29  1047.74  1026.25  1044.82   4062350.0   4.541328   
..           ...      ...      ...      ...      ...         ...        ...   
995  NSE:DRREDDY  1362.00  1367.00  1330.30  1341.35   3740109.0  17.364529   
996  NSE:DRREDDY  1347.00  1365.00  1336.20  1350.90   2587142.0  21.468928   
997  NSE:DRREDDY  1350.90  1360.20  1342.75  1355.15   1531359.0  24.778997   
998  NSE:DRREDDY  1363.05  1398.25  1345.50  1389.45   3656253.0  29.826156   
99

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: nan - val_loss: 0.0343
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: nan - val_loss: 0.0344
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: nan - val_loss: 0.0344
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: nan - val_loss: 0.0346
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: nan - val_loss: 0.0340
[[0.00283 ]
 [0.003937]
 [0.00475 ]
 [0.005714]
 [0.006874]
 [0.007812]
 [0.00865 ]
 [0.00908 ]
 [0.00916 ]
 [0.00953 ]
 [0.00967 ]
 [0.00924 ]
 [0.00836 ]
 [0.007248]
 [0.006504]
 [0.00603 ]
 [0.00571 ]
 [0.005554]
 [0.00546 ]
 [0.00514 ]
 [0.005142]
 [0.004997]
 [0.004963]
 [0.004707]
 [0.004337]
 [0.003914]
 [0.003555]
 [0.003218]
 [0.00311 ]
 [0.002886]
 [0.002638]
 [0.002863]
 [0.003258]
 [0.003975]
 [0.004765]
 [0.005882]
 [0.006783]
 [0.007454]
 [0.00805 ]
 [0.008354]
 [0.00841 ]
 [0.00827 ]
 [0.00803 ]
 [0.00825 ]
 [0.00877 ]
 [0.00953 ]
 [0.01037 ]
 [0.01126 ]
 [0.01194 ]
 [0.01218 ]
 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: nan - val_loss: 0.0023
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: 3.1683e-04
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: nan - val_loss: 1.6231e-04
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: 3.0531e-04
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: 1.5433e-04
[[0.0406 ]
 [0.04108]
 [0.04144]
 [0.04245]
 [0.0439 ]
 [0.04578]
 [0.0479 ]
 [0.05008]
 [0.05194]
 [0.05322]
 [0.05426]
 [0.05475]
 [0.05447]
 [0.05377]
 [0.0531 ]
 [0.05234]
 [0.05173]
 [0.05075]
 [0.0495 ]
 [0.04803]
 [0.0463 ]
 [0.0441 ]
 [0.042  ]
 [0.04034]
 [0.0389 ]
 [0.0381 ]
 [0.03754]
 [0.037  ]
 [0.03632]
 [0.0356 ]
 [0.03488]
 [0.03424]
 [0.03372]
 [0.03336]
 [0.0339 ]
 [0.03494]
 [0.0358 ]
 [0.03644]
 [0.03757]
 [0.0388 ]
 [0.03955]
 [0.04028]
 [0.04092]
 [0.0416 ]
 [0.04224]
 [0.04337]
 [0.04477]
 [0.0457 ]
 [0.04623]
 [0.04617]
 [0.0459 ]
 [0.04565]
 [0.0454 ]
 [0.0

...

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: nan - val_loss: 0.0115
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: nan - val_loss: 0.0114
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: 0.0117
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: 0.0114
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: 0.0114
[[-0.0226 ]
 [-0.0223 ]
 [-0.0221 ]
 [-0.02197]
 [-0.022  ]
 [-0.02213]
 [-0.02232]
 [-0.02257]
 [-0.0229 ]
 [-0.02327]
 [-0.02362]
 [-0.024  ]
 [-0.02435]
 [-0.02457]
 [-0.02463]
 [-0.02472]
 [-0.02473]
 [-0.02483]
 [-0.02496]
 [-0.02518]
 [-0.02542]
 [-0.0257 ]
 [-0.02603]
 [-0.02629]
 [-0.02649]
 [-0.02652]
 [-0.02641]
 [-0.0262 ]
 [-0.02586]
 [-0.02576]
 [-0.02582]
 [-0.02597]
 [-0.02621]
 [-0.0265 ]
 [-0.02693]
 [-0.02748]
 [-0.02817]
 [-0.02882]
 [-0.02937]
 [-0.02977]
 [-0.02992]
 [-0.02979]
 [-0.02945]
 [-0.02895]
 [-0.02823]
 [-0.02744]
 [-0.0268 ]
 [-0.02641]
 [-0.02617]
 [-0.0261 ]
 [-0

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: nan - val_loss: 0.0352
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: 0.0346
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: nan - val_loss: 0.0345
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: 0.0344
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: 0.0346
[[-0.0321 ]
 [-0.03308]
 [-0.03384]
 [-0.03452]
 [-0.03506]
 [-0.03537]
 [-0.03543]
 [-0.03525]
 [-0.03473]
 [-0.03394]
 [-0.0329 ]
 [-0.03174]
 [-0.03041]
 [-0.02908]
 [-0.02798]
 [-0.02711]
 [-0.02644]
 [-0.02583]
 [-0.02525]
 [-0.02487]
 [-0.02469]
 [-0.02464]
 [-0.02475]
 [-0.0251 ]
 [-0.02544]
 [-0.02591]
 [-0.0265 ]
 [-0.02707]
 [-0.02754]
 [-0.02791]
 [-0.02907]
 [-0.03062]
 [-0.03217]
 [-0.0336 ]
 [-0.03574]
 [-0.03796]
 [-0.04044]
 [-0.04282]
 [-0.0447 ]
 [-0.0467 ]
 [-0.0488 ]
 [-0.0509 ]
 [-0.05255]
 [-0.0541 ]
 [-0.0556 ]
 [-0.05713]
 [-0.05786]
 [-0.05814]
 [-0.05807]
 [-0.05707]
 [

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: nan - val_loss: 0.0126
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: nan - val_loss: 0.0115
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: nan - val_loss: 0.0113
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: nan - val_loss: 0.0113
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: nan - val_loss: 0.0115
[[0.02156]
 [0.02127]
 [0.02107]
 [0.02141]
 [0.02184]
 [0.02284]
 [0.02379]
 [0.02502]
 [0.02644]
 [0.02771]
 [0.02911]
 [0.02986]
 [0.0305 ]
 [0.03044]
 [0.03009]
 [0.02974]
 [0.02942]
 [0.02919]
 [0.02905]
 [0.02832]
 [0.02763]
 [0.02678]
 [0.02615]
 [0.02531]
 [0.02414]
 [0.02321]
 [0.02216]
 [0.02142]
 [0.02107]
 [0.02081]
 [0.02054]
 [0.02046]
 [0.02045]
 [0.02043]
 [0.02036]
 [0.02022]
 [0.01999]
 [0.01984]
 [0.01976]
 [0.0196 ]
 [0.01945]
 [0.01947]
 [0.01978]
 [0.01993]
 [0.02026]
 [0.02065]
 [0.02106]
 [0.0216 ]
 [0.02182]
 [0.02182]
 [0.02237]
 [0.02301]
 [0.0236 ]
 [0.02414]
 [0.025